<a href="https://colab.research.google.com/github/kakaandriano0-hash/natural-style/blob/main/tugas11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U transformers datasets accelerate evaluate scikit-learn pandas

import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [2]:
train_path = "dataset_sentimen_ml_300_unique_semantic.csv"
test_path  = "dataset_ml_400_unlabeled_playstore_style.csv"

df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

# Validasi kolom wajib
assert "komentar" in df_train.columns, "Train CSV harus punya kolom 'komentar'"
assert "label" in df_train.columns, "Train CSV harus punya kolom 'label'"
assert "komentar" in df_test.columns, "Test CSV harus punya kolom 'komentar'"

# Bersihin NaN dan strip
df_train = df_train.dropna(subset=["komentar", "label"]).copy()
df_train["komentar"] = df_train["komentar"].astype(str).str.strip()
df_train["label"] = df_train["label"].astype(str).str.strip().str.lower()

df_test = df_test.dropna(subset=["komentar"]).copy()
df_test["komentar"] = df_test["komentar"].astype(str).str.strip()

print(df_train.head())
print(df_train["label"].value_counts())
print("Test rows:", len(df_test))


                                            komentar    label
0  Sumpah nagih main solo, meta-nya lumayan seimb...  positif
1  Keren sih main bareng squad, meta-nya lumayan ...  positif
2  Worth it banget push rank di ML, animasi skill...  positif
3  Seru banget main solo, animasi skill nya kelia...  positif
4  Sumpah nagih main solo, meta-nya lumayan seimb...  positif
label
positif    100
negatif    100
netral     100
Name: count, dtype: int64
Test rows: 400


In [3]:
label2id = {"negatif": 0, "netral": 1, "positif": 2}
id2label = {v: k for k, v in label2id.items()}

# pastikan semua label valid
unknown = sorted(set(df_train["label"]) - set(label2id.keys()))
assert len(unknown) == 0, f"Ada label tidak dikenal: {unknown}"

df_train["labels"] = df_train["label"].map(label2id).astype(int)

train_df, val_df = train_test_split(
    df_train[["komentar", "labels"]],
    test_size=0.2,
    random_state=42,
    stratify=df_train["labels"]
)

print("Train:", len(train_df), "Val:", len(val_df))
print("Train dist:\n", train_df["labels"].value_counts())
print("Val dist:\n", val_df["labels"].value_counts())


Train: 240 Val: 60
Train dist:
 labels
0    80
1    80
2    80
Name: count, dtype: int64
Val dist:
 labels
0    20
2    20
1    20
Name: count, dtype: int64


In [4]:
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

model_ckpt = "indobenchmark/indobert-base-p1"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

hf_train = Dataset.from_pandas(train_df.reset_index(drop=True))
hf_val   = Dataset.from_pandas(val_df.reset_index(drop=True))
hf_test  = Dataset.from_pandas(df_test[["komentar"]].reset_index(drop=True))

def tokenize(batch):
    return tokenizer(
        batch["komentar"],
        truncation=True,
        max_length=160  # boleh 128-256
    )

hf_train = hf_train.map(tokenize, batched=True, remove_columns=["komentar"])
hf_val   = hf_val.map(tokenize, batched=True, remove_columns=["komentar"])
hf_test  = hf_test.map(tokenize, batched=True)  # jangan remove komentar, nanti buat output

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [5]:
import evaluate
from transformers import AutoModelForSequenceClassification

acc = evaluate.load("accuracy")
f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments, Trainer, set_seed

set_seed(42)

args = TrainingArguments(
    output_dir="ml_sentiment_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=6,              # kalau overfit turunkan 3-5
    weight_decay=0.01,
    eval_strategy="epoch",           # versi baru: eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    logging_steps=20,
    report_to="none",
    fp16=torch.cuda.is_available(),  # otomatis jika GPU
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()


/tmp/ipython-input-1255567612.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,0.123543,1.000000,1.000000
2,0.509000,0.009107,1.000000,1.000000
3,0.017300,0.003547,1.000000,1.000000
4,0.003900,0.002572,1.000000,1.000000
5,0.003900,0.002257,1.000000,1.000000
6,0.002900,0.002173,1.000000,1.000000


{'eval_loss': 0.12354329228401184,
 'eval_accuracy': 1.0,
 'eval_f1_macro': 1.0,
 'eval_runtime': 0.1325,
 'eval_samples_per_second': 452.867,
 'eval_steps_per_second': 15.096,
 'epoch': 6.0}

In [7]:
pred_out = trainer.predict(hf_test.remove_columns(["komentar"]))  # prediksi tanpa kolom teks
logits = pred_out.predictions
probs = torch.softmax(torch.tensor(logits), dim=-1).cpu().numpy()

pred_ids = probs.argmax(axis=1)
pred_labels = [id2label[i] for i in pred_ids]
confidence = probs.max(axis=1)

df_result = df_test.copy()
df_result["label_pred"] = pred_labels
df_result["confidence"] = np.round(confidence, 4)

df_result.head(10)


,komentar,label_pred,confidence
0,"Yang aku rasain mode classic, butuh koneksi in...",netral,0.8249
1,"Kalau dari pengalamanku game MOBA ini, hasil p...",netral,0.9047
2,"Setelah update kemarin game ini, permainannya ...",netral,0.7989
3,"Kalau main lama-lama sistem di ML, kadang juga...",negatif,0.6591
4,"Pas nyobain lagi Mobile Legends, kontrolnya cu...",netral,0.5511
5,"Yang aku rasain game MOBA ini, update-nya cuku...",netral,0.8914
6,"Kalau mabar mode ranked, kadang ketemu tim ena...",negatif,0.5689
7,Beberapa hari main game Mobile Legends Bang Ba...,netral,0.5864
8,"Menurutku ML, kerja sama tim jadi faktor utama...",netral,0.8718
9,"Sejauh ini game ini, banyak hero yang bisa dic...",netral,0.8910


In [8]:
out_path = "hasil_prediksi_400_transformer.csv"
df_result.to_csv(out_path, index=False)

from google.colab import files
files.download(out_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>